In [292]:
# TODO

# 1. computerTotalFantasyPoints()

In [1]:
import gazpacho as gp
import pandas as pd
from gazpacho import Soup, get

import playerposition as pp

In [ ]:
# CBS Data
positionInput = input(f"Input one of: F, C, W, D, G")

position = positionInput.upper()

baseUrl = f"https://www.cbssports.com/fantasy/hockey/stats"
url = f"{baseUrl}/{position}/2021/restofseason/projections"

print(f"URL: {url}")

html = get(url)

In [35]:
# League Scoring Settings

pointValues = {
    "G": 1.0,
    "A": 0.5,
    "PTS": 0.5,
    "SOG": 0.2,
    "STP": 0.5,
    "DEF": 0.5,
    "W": 3.0,
    "SV": 0.2,
    "GA": -0.5,
    "OTL": 1.0,
}

In [ ]:
# Invoke this cell if you'd like to change the scoring values of your category


def isFloat(checkInput: str):
    if "." in checkInput:
        split_number = checkInput.split(".")
        if (
            len(split_number) == 2
            and split_number[0].isdigit()
            and split_number[1].isdigit()
        ):
            return True
    return False


# category is one of: G, A, PTS, SOG, STP, DEF, W, SV, GA, OTL
def changeScoringValuesHelper(category: str):
    categoryKey = category.upper()
    if categoryKey not in pointValues.keys():
        return
    categoryVal = input(f"Value of {categoryKey}")
    if isFloat(categoryVal):
        try:
            pointValues[categoryKey] = (
                float(categoryVal) if categoryKey != "GA" else (float(categoryVal) * -1)
            )
            print(f"Successfully updated value of {categoryKey}")
        except:
            print(f"Value not updated")
            pass


def changeScoringValues():
    print(f"Please enter all values as floats (e.g. 1.0)")
    print(f"To skip a category, press ENTER")
    changeScoringValuesHelper("G")
    changeScoringValuesHelper("A")
    changeScoringValuesHelper("PTS")
    changeScoringValuesHelper("SOG")
    changeScoringValuesHelper("STP")
    changeScoringValuesHelper("DEF")
    changeScoringValuesHelper("W")
    changeScoringValuesHelper("SV")
    changeScoringValuesHelper("GA")
    changeScoringValuesHelper("OTL")


changeScoringValues()
print(pointValues)

In [ ]:
# Hockey-Reference Advanced Stats

ASData = pd.read_csv(f"./data/NHLAdvancedStats_Players2122.csv")

# Parsing Advanced Stats Data
advancedDF = pd.DataFrame(ASData)
cols = [3, 19, 20, 21, 22, 23]
advancedDF = advancedDF[advancedDF.columns[cols]]
print(advancedDF)

In [ ]:
# Parsing of HTML and extracting 'TableBase-bodyTr' tag
soup = Soup(html)
rows = soup.find("tr", {"class": "TableBase-bodyTr"})
headers = soup.find("tr", {"class": "TableBase-headTr"})

In [ ]:
# Data to Table (Players)


def parse_header(header, pos: str):
    player = header.find("a")[0].text.upper().strip()
    position = "POS"
    team = "TEAM"

    # Skaters
    if pos != "G":
        goalsScored = header.find("a")[4].text.upper()
        assists = header.find("a")[5].text.upper()
        points = header.find("a")[6].text.upper()
        shotsOnGoal = header.find("a")[15].text.upper()
        return player, team, position, goals, assists, points, shotsOnGoal
    # Goalies
    else:
        wins = header.find("a")[5].text.upper()
        shutouts = header.find("a")[7].text.upper()
        goalsAgainst = header.find("a")[8].text.upper()
        shotsAgainst = header.find("a")[10].text.upper()
        saves = header.find("a")[11].text.upper()
        return player, team, position, wins, shutouts, goalsAgainst, shotsAgainst, saves


def parse_row(row, pos: str):
    # currPlayer is its own class "CellPlayerName--long" in 'TableBase-bodyTr'
    # Find name, position, team for currPlayer
    currPlayer = row.find("span", {"class": "CellPlayerName--long"})
    name = currPlayer.find("a").text
    position = currPlayer.find("span", {"class": "CellPlayerName-position"}).text
    team = currPlayer.find("span", {"class": "CellPlayerName-team"}).text

    # Skaters
    if pos != "G":
        goalsScored = int(row.find("td", {"class": "TableBase-bodyTd"})[4].text)
        assists = int(row.find("td", {"class": "TableBase-bodyTd"})[5].text)
        points = int(row.find("td", {"class": "TableBase-bodyTd"})[6].text)
        shotsOnGoal = int(row.find("td", {"class": "TableBase-bodyTd"})[15].text)
        return name, team, position, goals, assists, points, shotsOnGoal
    # Goalies
    else:
        wins = row.find("td", {"class": "TableBase-bodyTd"})[5].text.strip()
        shutouts = row.find("td", {"class": "TableBase-bodyTd"})[7].text.strip()
        goalsAgainst = int(row.find("td", {"class": "TableBase-bodyTd"})[8].text)
        saves = int(row.find("td", {"class": "TableBase-bodyTd"})[10].text)
        shotsAgainst = int(row.find("td", {"class": "TableBase-bodyTd"})[11].text)

        return name, team, position, wins, shutouts, goalsAgainst, shotsAgainst, saves


playerInfoDF = pd.DataFrame(
    [parse_row(row, position) for row in rows], columns=parse_header(headers, position)
)
# print(playerInfoDF.columns)
# print(advancedDF.columns)
# print(playerInfoDF.columns.intersection(advancedDF.columns))


if position != "G":
    result = pd.merge(playerInfoDF, advancedDF, on="PLAYER")
else:
    result = playerInfoDF
print(result)